In [ ]:
 import pandas as pd

data = pd.read_csv("/content/tr_q1_pair_with_similarity_cos.csv")

In [ ]:
data2 = data[["right_ans","student_ans"]]
data2

In [ ]:
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
import re

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def make_lower_case(text):
    return text.lower()

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

data2['cleaned'] = data2['student_ans'].apply(make_lower_case)
data2['cleaned'] = data2.cleaned.apply(remove_stop_words)
data2['cleaned'] = data2.cleaned.apply(remove_html)


In [ ]:
data2

In [ ]:
senlist = []
for sentence in data2["cleaned"]:
  a = sentence.replace("\ufeff", "")
  senlist.append(a)

In [ ]:
senlist

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

commentlist=[]

for sentence in data2["student_ans"]:

  word_tokens = word_tokenize(sentence)

  result = []

  for w in word_tokens:

    result.append(w)

  commentlist.append(result)

commentlist

In [ ]:
commentlist_cleaned=[]

for sentence2 in data2["cleaned"]:

  word_tokens2 = word_tokenize(sentence2)

  result_cleaned = []

  for w2 in word_tokens2:

    result_cleaned.append(w2)

  commentlist_cleaned.append(result_cleaned)

commentlist_cleaned

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=commentlist, size=100, window=5, min_count=0, workers=4, sg=0)
model_cleaned = Word2Vec(sentences=commentlist_cleaned, size=100, window=5, min_count=0, workers=4, sg=0)

In [ ]:
model_result = model.wv.most_similar("parallel")
print(model_result)

In [ ]:
model_cleaned_result = model_cleaned.wv.most_similar("parallel")
print(model_cleaned_result)

In [ ]:
# 단어벡터의 평균 구하기

In [ ]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model_cleaned.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model_cleaned[word]
                else:
                    doc2vec = doc2vec + model_cleaned[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [ ]:
get_document_vectors(data2['cleaned'])